In [2]:
import pandas as pd
# from pandas import datetime
from pandas import read_csv
from pandas import DataFrame

import numpy as np

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from pylab import rcParams

from pmdarima.arima import auto_arima

import matplotlib.pyplot as plt

import yfinance as yf

from datetime import datetime
from datetime import timedelta

import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'datetime' from 'pandas' (c:\NUS\Fintech-Hacka\venv\lib\site-packages\pandas\__init__.py)

In [ ]:
#getting data
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
eth_df = yf.download('ETH-USD',start_date, today)
eth_df.tail()

# Preprocess data
dataset_ex_df = eth_df.copy()
dataset_ex_df = dataset_ex_df.reset_index()
dataset_ex_df['Date'] = pd.to_datetime(dataset_ex_df['Date'])
dataset_ex_df.set_index('Date', inplace=True)
dataset_ex_df = dataset_ex_df['Close'].to_frame()

#plot close price
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Close Prices')
plt.plot(dataset_ex_df['Close'])
plt.title('ETH closing price')
plt.show()

In [ ]:
#Distribution of the dataset
df_close = dataset_ex_df['Close']
plt.title('ETH price distribution')
df_close.plot(kind='kde')

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
test_stationarity(df_close)

In [ ]:
#To separate the trend and the seasonality from a time series, 
# we can decompose the series using the following code.
result = seasonal_decompose(df_close, model='multiplicative')
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(16, 9)


In [ ]:
#if not stationary then eliminate trend
#Eliminate trend

rcParams['figure.figsize'] = 10, 6
df_log = np.log(df_close)
moving_avg = df_log.rolling(12).mean()
std_dev = df_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()


In [ ]:
#split data into train and training set
train_data, test_data = df_log[3:int(len(df_close)*0.95)], df_log[int(len(df_log)*0.95):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.plot(df_log, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())

In [ ]:
# Build Model
model = ARIMA(train_data, order=(2,1,1))  
fitted = model.fit()
print(fitted.summary())

In [ ]:
df = dataset_ex_df
model_future = ARIMA(df,order=(2, 1, 1))
results_future = model_future.fit()
predictions_future = results_future.predict(len(df),len(df)+50,typ = 'levels')
df.plot(legend=True , figsize = (12,8))
predictions_future.plot(legend=True)

print(df[-2:])
print(predictions_future[:2])